In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
try:
    train_df = pd.read_csv("train.csv")
    test_df = pd.read_csv("test.csv")
    print("Arquivos 'train.csv' e 'test.csv' carregados com sucesso!")
except FileNotFoundError:
    print("\nERRO: Arquivo não encontrado!")

Arquivos 'train.csv' e 'test.csv' carregados com sucesso!


In [3]:
print("Preparando dados e selecionando as melhores features...")
features_to_drop = ['labels', 'id', 'category_code']
X = train_df.drop(columns=features_to_drop)
y = train_df['labels']

# Análise de importância para encontrar as melhores features
temp_rf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1))
])
temp_rf.fit(X, y)
importances = temp_rf.named_steps['classifier'].feature_importances_
feature_importance_df = pd.DataFrame({'feature': X.columns, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

N_TOP_FEATURES = 15
top_features = feature_importance_df['feature'].head(N_TOP_FEATURES).tolist()
X_selected = X[top_features]

print(f"Otimização será feita com as {N_TOP_FEATURES} features mais importantes.")

Preparando dados e selecionando as melhores features...
Otimização será feita com as 15 features mais importantes.


In [4]:
print("\n--- Etapa de Otimização (GridSearchCV) nas Features Selecionadas ---")

pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1))
])

# Grade de parâmetros um pouco mais focada
param_grid = {
    'classifier__n_estimators': [200, 300],
    'classifier__max_depth': [8, 10, 12],
    'classifier__min_samples_leaf': [2, 4],
    'classifier__max_features': ['sqrt', 0.5] # Adicionando mais um parâmetro para testar
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=1)

print("Iniciando a busca... Isso pode demorar alguns minutos.")
grid_search.fit(X_selected, y)

print("\n--- RESULTADO DA OTIMIZAÇÃO ---")
print(f"Melhor Acurácia de Validação Cruzada: {grid_search.best_score_:.4f}")
print("Melhores Parâmetros Encontrados:")
print(grid_search.best_params_)
print("---------------------------------")


--- Etapa de Otimização (GridSearchCV) nas Features Selecionadas ---
Iniciando a busca... Isso pode demorar alguns minutos.
Fitting 5 folds for each of 24 candidates, totalling 120 fits

--- RESULTADO DA OTIMIZAÇÃO ---
Melhor Acurácia de Validação Cruzada: 0.7894
Melhores Parâmetros Encontrados:
{'classifier__max_depth': 10, 'classifier__max_features': 0.5, 'classifier__min_samples_leaf': 2, 'classifier__n_estimators': 200}
---------------------------------


In [5]:
print("\n--- Etapa de Submissão ---")
print("Usando o melhor modelo encontrado para gerar as previsões finais...")
best_model = grid_search.best_estimator_

# Prepara o test set com as mesmas features
test_ids = test_df['id']
X_test_selected = test_df[top_features]
X_selected, X_test_selected = X_selected.align(X_test_selected, join='left', axis=1, fill_value=0)

final_predictions = best_model.predict(X_test_selected)
submission_df = pd.DataFrame({'id': test_ids, 'labels': final_predictions})
submission_df.to_csv('final_submission_optimized_selected_features.csv', index=False)

print("\nArquivo 'final_submission_optimized_selected_features.csv' criado com sucesso!")


--- Etapa de Submissão ---
Usando o melhor modelo encontrado para gerar as previsões finais...

Arquivo 'final_submission_optimized_selected_features.csv' criado com sucesso!
